<a href="https://colab.research.google.com/github/KathituCodes/2025-HEINEKEN-CHINESE-GRAND-PRIX-Predictions./blob/main/2025_HEINEKEN_CHINESE_GRAND_PRIX_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install fastf1

In [18]:
# Load the necessary Libraries
import os
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [19]:
#Create cache to avoid redownloading the FastF1 data during each run
cache_dir = "f1_cache"

# Create the directory if it does not exist
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

# Enable caching
fastf1.Cache.enable_cache(cache_dir)

print("Cache enabled successfully!")

Cache enabled successfully!


# **Load historical race data**
- Load the 2024 China Grand Prix race session (5rd race of 2024)
- Examine the session results data structure

In [20]:
import fastf1

# Load the Chinese GP session (likely the 5th race of the 2024 season)
session_2024_china = fastf1.get_session(2024, 5, "R")
session_2024_china.load()

# Display session info
print(session_2024_china.results)  # Check race results


core           INFO 	Loading data for Chinese Grand Prix - Race [v3.5.3]
INFO:fastf1.fastf1.core:Loading data for Chinese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
INFO:fastf1.fastf1.req:Using cached data for timing_app_data
core         

   DriverNumber BroadcastName Abbreviation         DriverId         TeamName  \
1             1  M VERSTAPPEN          VER   max_verstappen  Red Bull Racing   
4             4      L NORRIS          NOR           norris          McLaren   
11           11       S PEREZ          PER            perez  Red Bull Racing   
16           16     C LECLERC          LEC          leclerc          Ferrari   
55           55       C SAINZ          SAI            sainz          Ferrari   
63           63     G RUSSELL          RUS          russell         Mercedes   
14           14      F ALONSO          ALO           alonso     Aston Martin   
81           81     O PIASTRI          PIA          piastri          McLaren   
44           44    L HAMILTON          HAM         hamilton         Mercedes   
27           27  N HULKENBERG          HUL       hulkenberg     Haas F1 Team   
31           31        E OCON          OCO             ocon           Alpine   
23           23       A ALBON          A

# **Prepare 2025 qualifying data**

- Create a DataFrame with 2025 qualifying information (drivers, qualifying times, grid positions)
- Convert qualifying lap times from string format (MM.sss) to seconds
- Map full driver names to FastF1 three-letter codes

In [21]:
#session_2024_china.weather()
session_2024_china.results.info()
session_2024_china.results.head(15)

<class 'fastf1.core.SessionResults'>
Index: 20 entries, 1 to 77
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   DriverNumber        20 non-null     object         
 1   BroadcastName       20 non-null     object         
 2   Abbreviation        20 non-null     object         
 3   DriverId            20 non-null     object         
 4   TeamName            20 non-null     object         
 5   TeamColor           20 non-null     object         
 6   TeamId              20 non-null     object         
 7   FirstName           20 non-null     object         
 8   LastName            20 non-null     object         
 9   FullName            20 non-null     object         
 10  HeadshotUrl         20 non-null     object         
 11  CountryCode         20 non-null     object         
 12  Position            20 non-null     float64        
 13  ClassifiedPosition  20 non-null     object 

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,NED,1.0,1,1.0,NaT,NaT,NaT,0 days 01:40:52.554000,Finished,25.0
4,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,GBR,2.0,2,4.0,NaT,NaT,NaT,0 days 00:00:13.773000,Finished,18.0
11,11,S PEREZ,PER,perez,Red Bull Racing,3671C6,red_bull,Sergio,Perez,Sergio Perez,...,MEX,3.0,3,2.0,NaT,NaT,NaT,0 days 00:00:19.160000,Finished,15.0
16,16,C LECLERC,LEC,leclerc,Ferrari,E80020,ferrari,Charles,Leclerc,Charles Leclerc,...,MON,4.0,4,6.0,NaT,NaT,NaT,0 days 00:00:23.623000,Finished,12.0
55,55,C SAINZ,SAI,sainz,Ferrari,E80020,ferrari,Carlos,Sainz,Carlos Sainz,...,ESP,5.0,5,7.0,NaT,NaT,NaT,0 days 00:00:33.983000,Finished,10.0
63,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,GBR,6.0,6,8.0,NaT,NaT,NaT,0 days 00:00:38.724000,Finished,8.0
14,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,ESP,7.0,7,3.0,NaT,NaT,NaT,0 days 00:00:43.414000,Finished,7.0
81,81,O PIASTRI,PIA,piastri,McLaren,FF8000,mclaren,Oscar,Piastri,Oscar Piastri,...,AUS,8.0,8,5.0,NaT,NaT,NaT,0 days 00:00:56.198000,Finished,4.0
44,44,L HAMILTON,HAM,hamilton,Mercedes,27F4D2,mercedes,Lewis,Hamilton,Lewis Hamilton,...,GBR,9.0,9,18.0,NaT,NaT,NaT,0 days 00:00:57.986000,Finished,2.0
27,27,N HULKENBERG,HUL,hulkenberg,Haas F1 Team,B6BABD,haas,Nico,Hulkenberg,Nico Hulkenberg,...,GER,10.0,10,9.0,NaT,NaT,NaT,0 days 00:01:00.476000,Finished,1.0


In [22]:
import pandas as pd

# 2025 HEINEKEN CHINESE GRAND PRIX 2025
qualifying_2025 = pd.DataFrame({
    "Position": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "Driver": [
        "Oscar Piastri", "George Russell", "Lando Norris", "Max Verstappen", "Lewis Hamilton",
        "Charles Leclerc", "Isack Hadjar", "Andrea Kimi Antonelli", "Yuki Tsunoda", "Alex Albon"
    ],
    "Team": [
        "McLaren", "Mercedes", "McLaren", "Red Bull", "Ferrari",
        "Ferrari", "AlphaTauri", "Mercedes", "Racing Bulls", "Williams"
    ],
    "Qualifying Time": [
        "1:33.018", "1:33.102", "1:33.201", "1:33.289", "1:33.415",
        "1:33.523", "1:33.678", "1:33.755", "1:33.862", "1:33.971"
    ]
})

print(qualifying_2025)


   Position                 Driver          Team Qualifying Time
0         1          Oscar Piastri       McLaren        1:33.018
1         2         George Russell      Mercedes        1:33.102
2         3           Lando Norris       McLaren        1:33.201
3         4         Max Verstappen      Red Bull        1:33.289
4         5         Lewis Hamilton       Ferrari        1:33.415
5         6        Charles Leclerc       Ferrari        1:33.523
6         7           Isack Hadjar    AlphaTauri        1:33.678
7         8  Andrea Kimi Antonelli      Mercedes        1:33.755
8         9           Yuki Tsunoda  Racing Bulls        1:33.862
9        10             Alex Albon      Williams        1:33.971


In [23]:
# Convert lap times to seconds
def convert_time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    minutes, seconds = map(float, time_str.split(":"))
    return minutes * 60 + seconds

# Apply the function to the correct column ('Qualifying Time')
qualifying_2025["QualifyingTime (s)"] = qualifying_2025["Qualifying Time"].apply(convert_time_to_seconds)
# Drop original column (renamed to match the existing column name)
qualifying_2025.drop(columns=["Qualifying Time"], inplace=True)

qualifying_2025

,Position,Driver,Team,QualifyingTime (s)
0,1,Oscar Piastri,McLaren,93.018
1,2,George Russell,Mercedes,93.102
2,3,Lando Norris,McLaren,93.201
3,4,Max Verstappen,Red Bull,93.289
4,5,Lewis Hamilton,Ferrari,93.415
5,6,Charles Leclerc,Ferrari,93.523
6,7,Isack Hadjar,AlphaTauri,93.678
7,8,Andrea Kimi Antonelli,Mercedes,93.755
8,9,Yuki Tsunoda,Racing Bulls,93.862
9,10,Alex Albon,Williams,93.971


In [24]:
# Map full names to FastF1 3-letter codes
driver_mapping = { "Oscar Piastri": "PIA", "George Russell": "RUS", "Lando Norris": "NOR", "Max Verstappen": "VER",
                   "Lewis Hamilton": "HAM", "Charles Leclerc": "LEC", "Isack Hadjar": "HAD", "Andrea Kimi Antonelli": "ANT",
                   "Yuki Tsunoda": "TSU",  "Alexander Albon": "ALB"
}

qualifying_2025["DriverCode"] = qualifying_2025["Driver"].map(driver_mapping)

qualifying_2025

,Position,Driver,Team,QualifyingTime (s),DriverCode
0,1,Oscar Piastri,McLaren,93.018,PIA
1,2,George Russell,Mercedes,93.102,RUS
2,3,Lando Norris,McLaren,93.201,NOR
3,4,Max Verstappen,Red Bull,93.289,VER
4,5,Lewis Hamilton,Ferrari,93.415,HAM
5,6,Charles Leclerc,Ferrari,93.523,LEC
6,7,Isack Hadjar,AlphaTauri,93.678,HAD
7,8,Andrea Kimi Antonelli,Mercedes,93.755,ANT
8,9,Yuki Tsunoda,Racing Bulls,93.862,TSU
9,10,Alex Albon,Williams,93.971,NaN


In [25]:
session_2024_china.laps.info()

<class 'fastf1.core.Laps'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   Time                1032 non-null   timedelta64[ns]
 1   Driver              1032 non-null   object         
 2   DriverNumber        1032 non-null   object         
 3   LapTime             1007 non-null   timedelta64[ns]
 4   LapNumber           1032 non-null   float64        
 5   Stint               1032 non-null   float64        
 6   PitOutTime          41 non-null     timedelta64[ns]
 7   PitInTime           41 non-null     timedelta64[ns]
 8   Sector1Time         1009 non-null   timedelta64[ns]
 9   Sector2Time         1030 non-null   timedelta64[ns]
 10  Sector3Time         1030 non-null   timedelta64[ns]
 11  Sector1SessionTime  1009 non-null   timedelta64[ns]
 12  Sector2SessionTime  1030 non-null   timedelta64[ns]
 13  Sector3SessionTime  1030 non-null   timedelt

# **Process 2024 race data**

- Extract lap times, lap numbers, and tire compounds from the 2024 session
- Convert lap times to seconds
- Extract grid positions from 2024 race results

## **Merge datasets**

- Combine 2025 qualifying data with 2024 race data using driver codes as the key

In [26]:
# Extract lap times, grid position, lap number, and tire compound from 2024 session
laps_2024 = session_2024_china.laps[["Driver", "LapTime", "LapNumber", "Compound"]].copy()

# Convert LapTime to seconds
laps_2024.dropna(subset=["LapTime"], inplace=True)
laps_2024["LapTime (s)"] = laps_2024["LapTime"].dt.total_seconds()

# Extract grid position from 2024 race results
grid_positions_2024 = session_2024_china.results[["Abbreviation", "GridPosition"]].copy()
grid_positions_2024.rename(columns={"GridPosition": "GridPosition_2024"}, inplace=True)  # Rename column for clarity

# Merge lap times with grid position from last year's race
laps_2024 = laps_2024.merge(grid_positions_2024, left_on="Driver", right_on="Abbreviation", how="left")
laps_2024.drop(columns=["Abbreviation"], inplace=True)  # Drop duplicate driver column

# Merge 2025 Qualifying Data with 2024 Race Data
merged_data = qualifying_2025.merge(laps_2024, left_on="DriverCode", right_on="Driver", how="left")

# Display the updated dataframe
merged_data.head()

# Remove rows with NaN values in 'LapTime (s)' before training
merged_data.dropna(subset=['LapTime (s)'], inplace=True)

merged_data.head()

,Position,Driver_x,Team,QualifyingTime (s),DriverCode,Driver_y,LapTime,LapNumber,Compound,LapTime (s),GridPosition_2024
0,1,Oscar Piastri,McLaren,93.018,PIA,PIA,0 days 00:01:45.121000,1.0,MEDIUM,105.121,5.0
1,1,Oscar Piastri,McLaren,93.018,PIA,PIA,0 days 00:01:41.607000,2.0,MEDIUM,101.607,5.0
2,1,Oscar Piastri,McLaren,93.018,PIA,PIA,0 days 00:01:41.754000,3.0,MEDIUM,101.754,5.0
3,1,Oscar Piastri,McLaren,93.018,PIA,PIA,0 days 00:01:42.247000,4.0,MEDIUM,102.247,5.0
4,1,Oscar Piastri,McLaren,93.018,PIA,PIA,0 days 00:01:42.227000,5.0,MEDIUM,102.227,5.0


# **Build and train prediction model**

- Prepare features (qualifying time, grid position) and target variable (lap time)
- Split data into training and testing sets
- Train a Gradient Boosting Regressor model

**Make predictions for 2025:**

- Use the trained model to predict lap times for the 2025 Australian GP
- Display the predicted 2025 standings


**Evaluate model performance:**

- Calculate Mean Absolute Error (MAE) to assess prediction accuracy

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# The column name is 'Position', not 'GridPosition_2025'
X = merged_data[["QualifyingTime (s)", "Position"]]
y = merged_data["LapTime (s)"]

# Train Gradient Boosting Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)
model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, random_state=39)
model.fit(X_train, y_train)

# Predict using 2025 data
# Use 'Position' here as well to match the training data
predicted_lap_times = model.predict(qualifying_2025[["QualifyingTime (s)", "Position"]])
qualifying_2025["PredictedRaceTime (s)"] = predicted_lap_times

# Display final predictions
print("\n PREDICTED 2025 HEINEKEN CHINESE GRAND PRIX STANDINGS\n")
print(qualifying_2025[["Driver", "PredictedRaceTime (s)"]])

# Evaluate Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("\nModel Error (MAE): {:.2f} seconds".format(mae))


 PREDICTED 2025 HEINEKEN CHINESE GRAND PRIX STANDINGS

                  Driver  PredictedRaceTime (s)
0          Oscar Piastri             108.002228
1         George Russell             108.992479
2           Lando Norris             106.676945
3         Max Verstappen             105.229290
4         Lewis Hamilton             109.766347
5        Charles Leclerc             106.175762
6           Isack Hadjar             106.175762
7  Andrea Kimi Antonelli             111.684581
8           Yuki Tsunoda             111.684581
9             Alex Albon             111.684581

Model Error (MAE): 9.88 seconds
